In [1]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [2]:
# Gather Data
boston_dataset = load_boston()
data = pd.DataFrame(data= boston_dataset.data, columns=boston_dataset.feature_names)
features = data.drop(['INDUS', 'AGE'], axis=1)

log_prices = np.log(boston_dataset.target)
target = pd.DataFrame(log_prices, columns=['PRICE'])

In [22]:
CRIM_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

# property_stats = np.ndarray(shape=(1,11))
# property_stats[0][CRIME_IDX] = 0.02

property_stats = features.mean().values.reshape(1,11)

In [16]:
reg = LinearRegression().fit(features,target)
fitted_vals = reg.predict(features)
MSE = mean_squared_error(target, fitted_vals)
RMSE = np.sqrt(MSE)

In [29]:
def get_log_estimate(nm_rooms, student_per_classroom,
                    next_to_river=False,high_confidence=True):
    #Cofigure property
    property_stats[0][RM_IDX] = nm_rooms
    property_stats[0][PTRATIO_IDX] = student_per_classroom
    
    if next_to_river:
        property_stats[0][CHAS_IDX]=1
    else:
        property_stats[0][CHAS_IDX]=0
        
     #Make Predictions
    log_estimate = reg.predict(property_stats)[0][0]
    
    
    # Calc Range
    if high_confidence:
        upper_bound = log_estimate +2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68
    
   
    
    return log_estimate, upper_bound, lower_bound, interval

In [30]:
get_log_estimate(3,20)

(2.6716097430689403, 3.046634013463201, 2.2965854726746797, 95)

In [36]:
ZILLOW_MEDIAN_PRICE = 583.3
SCALE_FACTOR = ZILLOW_MEDIAN_PRICE / np.median(boston_dataset.target)

log_est, upper, lower, conf = get_log_estimate(9, student_per_classroom=15,
                                              next_to_river=False,
                                              high_confidence=False)

#Convert to today's dollars
dollar_est = np.e**log_est*1000*SCALE_FACTOR
dollar_hi = np.e**upper*1000*SCALE_FACTOR
dollar_low = np.e**lower*1000*SCALE_FACTOR

# ROund the dollar value to nearest thousand
rounded_est = np.around(dollar_est,-3)
rounded_hi = np.around(dollar_hi,-3)
rounded_low = np.around(dollar_low,-3)


print(f'The estimated property price is:{rounded_est}')
print(f'At {conf}% confidence the valuation range is')
print(f'between USD {rounded_hi} and USD {rounded_low}')

The estimated property price is:827000.0
At 68% confidence the valuation range is
between USD 997000.0 and USD 685000.0


In [51]:
def get_dollar_est(rm, ptratio, chas=False, large_range=True):
    """Estimate the price of a property in Boston.
    Keyword arguments:
    rm-- number of rooms in the property.
    ptratio--number of students per teacher in the classroom for the school in the area.
    chas -- True if the property is next to the river, False otherwise.
    large_range-- True for a 95% prediction interval, False for a 68% interval.
    
    
    """
    
    
    
    if rm < 1 or ptratio < 1:
        print('That is unrealistic TRY AGAIN!!')
        return
    
    log_est, upper, lower, conf = get_log_estimate(rm, student_per_classroom=ptratio,
                                              next_to_river=chas,
                                              high_confidence=large_range)

    #Convert to today's dollars
    dollar_est = np.e**log_est*1000*SCALE_FACTOR
    dollar_hi = np.e**upper*1000*SCALE_FACTOR
    dollar_low = np.e**lower*1000*SCALE_FACTOR

    # ROund the dollar value to nearest thousand
    rounded_est = np.around(dollar_est,-3)
    rounded_hi = np.around(dollar_hi,-3)
    rounded_low = np.around(dollar_low,-3)


    print(f'The estimated property price is:{rounded_est}')
    print(f'At {conf}% confidence the valuation range is')
    print(f'between USD {rounded_hi} and USD {rounded_low}')

In [46]:
get_dollar_est(3,0,chas=True)

That is unrealistic TRY AGAIN
